In [1]:
import torch
import numpy as np

In [2]:
EPS   = 0.001; # the regularization constant
sigma = 2**-16; # the kernel width
N     = 632;  # number of persons
d     = 100;  # number of features

In [3]:
N = 632
perm = torch.randperm(N)
print(torch.min(perm), torch.max(perm))

tensor(0) tensor(631)


In [4]:
idxtrain = perm[:N//2]
idxtest = perm[N//2:]
print(torch.min(idxtrain), torch.max(idxtrain))
print(torch.min(idxtest), torch.max(idxtest))

tensor(0) tensor(627)
tensor(5) tensor(631)


In [5]:
file_name = "viper//viper_features.npy"
mat = np.load(file_name, allow_pickle=True)
idxa = torch.from_numpy((mat.item().get('idxa')-1).astype('int64'))
idxb = torch.from_numpy((mat.item().get('idxb')-1).astype('int64'))

In [6]:
# first_ind is our set of similiaraity constraints so what we're doing is since idxtrain is a random permutation of 0:631 in 316 elements which act as indexes for the idxa thing and since we are talking 'bout Similaity pairwise constraint hence we're concatinating similiar element in pair e.g. let idxtrain = [[0, 345, 123]] be in R3 then idxa[0, idxtrain] = [x1, x2, x3] then first_ind = [[x1, x2, x3], [x1, x2, x3]] pairs. and since in D we want dissimmilar pairs hence we're randomly permutating them for the second phase of cat. e.g. second_ind = [idxb[[0, idxtrain]], idxb[0, randperm(idxtrain)]]

In [7]:
d = 100
X = torch.from_numpy(mat.item().get('ux'))[:d, :]

In [8]:
X.shape

torch.Size([100, 1264])

In [9]:
from demo import ret_logical, change_view
matches = ret_logical(N)

In [10]:
first_ind = change_view(torch.cat((idxa[0, idxtrain], idxa[0, idxtrain])))
second_ind = change_view(torch.cat((idxb[0, idxtrain], idxb[0, idxtrain[torch.randperm(N//2)]])))

In [11]:
S = torch.cat((first_ind[:, matches], second_ind[:, matches])) # must-link constraints
D = torch.cat((first_ind[:, torch.logical_not(matches)], second_ind[:, torch.logical_not(matches)])) # cannot-link constraints

In [12]:
torch.min(S[0,:]), torch.max(S[0,:]), torch.min(S[1,:]), torch.max(S[1,:])

(tensor(0), tensor(627), tensor(632), tensor(1259))

In [13]:
S[0, 0], S[1, 0]

(tensor(224), tensor(856))

In [14]:
from kernelmatrix import kernelmatrix
K = kernelmatrix(X, X2=torch.tensor([]), sigma=sigma)
print(K.shape)

torch.Size([1264, 1264])


In [15]:
from computeH import computeH
n = K.shape[0]
n1 = S.shape[1]
n0 = D.shape[1]
H0 = computeH(n, D[0, :], D[1, :])
H1 = computeH(n, S[0, :], S[1, :])

In [16]:
B = torch.zeros(n, 1)
E = torch.zeros(n, 1)
W = torch.zeros(n, n)
ib, ie = S[0, :], S[1, :]
o = torch.ones(ib.shape[0], 1)
B[ib] = o
E[ie] = o

In [17]:
def Calc_W(ib, ie, W):

    for i in range(0, ib.shape[0]):
        W[ib[i], ie[i]] = W[ib[i], ie[i]] + 1
        
    return W
W = Calc_W(ib, ie, W)
print(torch.max(ib), torch.max(ie), W[torch.max(ib), torch.max(ie)]
)

tensor(627) tensor(1259) tensor(1.)


### Caluclate $\Sigma_{1} = (\frac{1}{n1})XH_{1}X^T$

In [18]:
temp_a = torch.mm(X.double(), H1.double())
temp_b = torch.mm(temp_a, X.T)
Sigma1 = (1/S.shape[1])*temp_b

# Similarly calculate Sigma0
Sigma0 = (1/D.shape[1])*torch.mm(torch.mm(X.double(), H0.double()), X.T)

In [19]:
Sigma1_inv = torch.inverse(Sigma1)
Sigma0_inv = torch.inverse(Sigma0)
M_ = Sigma1_inv - Sigma0_inv

In [20]:
from helps.kernel import kernel
M = kernel(0.001, S, D, K, 1)
print(M.shape)

(1264, 1264)


In [21]:
# np.save('M.npy', M.numpy())
# np.save('idxa.npy', idxa.numpy())
# np.save('idxb.npy', idxb.numpy())
# np.save('idxtest.npy', idxtest.numpy())
# np.save('idxtrain.npy', idxtrain.numpy())

In [22]:
# it hints that if there is some problem it is inside computeT function because CalcMCMC is just doing one thing which is to calculate Mahalanobis distance, it might be that the problem is in CalcMCMC but it will be our last resort.

In [23]:
x = torch.Tensor([[0.169019, 0.958515, 0.162583], [0.466288, 0.794831, 0.375050], [0.407539, 0.010616, 0.373111]])
print(x.shape)
print(x.double())

torch.Size([3, 3])
tensor([[0.1690, 0.9585, 0.1626],
        [0.4663, 0.7948, 0.3751],
        [0.4075, 0.0106, 0.3731]], dtype=torch.float64)


In [24]:
L, V = torch.eig(x, eigenvectors=True)

In [25]:
print(L)

tensor([[ 1.3417,  0.0000],
        [-0.1323,  0.0000],
        [ 0.1276,  0.0000]])


In [26]:
print(V)

tensor([[-0.6312, -0.7725, -0.5098],
        [-0.7258,  0.1377, -0.1224],
        [-0.2735,  0.6199,  0.8516]])


In [27]:
torch.norm(V, dim=0)

tensor([1.0000, 1.0000, 1.0000])

In [28]:
torch.min(idxtest), torch.max(idxtest), torch.min(idxtrain), torch.max(idxtrain)

(tensor(5), tensor(631), tensor(0), tensor(627))

In [29]:
torch.min(idxa[:, idxtest]), torch.max(idxa[:, idxtest])

(tensor(5), tensor(631))

In [30]:
idxa = idxa.view(idxa.shape[1])
idxb = idxb.view(idxb.shape[1])

In [31]:
data = K
p = data[:, idxa[idxtest]]
q = data[:, idxb[idxtest]]

### Compute the Mahalanobis distance

In [32]:
A =  K
(d, pn) = p.shape
(d, qn) = q.shape
Ap = torch.mm(A.double(), p)
Aq = torch.mm(A.double(), q)
pmag = torch.sum(p*Ap, 0, keepdim=True).double()
qmag = torch.sum(q*Aq, 0, keepdim=True).double()

In [33]:
dist = torch.mm(torch.ones(pn, 1).double(), qmag) + torch.mm(pmag.T, torch.ones(1, qn).double()) - 2 * torch.mm(p.T, q)

In [34]:
result = torch.zeros(1, dist.shape[1])
for pairCounter in range(0, dist.shape[1]):
    distPair = dist[pairCounter, :]
    tmp, idx = torch.sort(distPair)
    result[:, idx==pairCounter] = result[:, idx==pairCounter] + 1

tmp = 0
for counter in range(1, result.shape[0]):
    result[:, counter] = result[:, counter] + tmp
    tmp = result[:, counter]

In [35]:
import numpy as np
np.linalg.eig(torch.rand(3,3).numpy())

(array([ 1.8613818 , -0.41706017,  0.04769045], dtype=float32),
 array([[-0.5295369 , -0.629563  , -0.01616446],
        [-0.5056032 ,  0.7722581 , -0.90522945],
        [-0.6811432 ,  0.0852519 ,  0.42461553]], dtype=float32))

In [36]:
x = [[0.870188 , 0.111200 , 0.185495],
   [0.791261 , 0.514741 , 0.037194],
   [0.392026 , 0.209612 , 0.270831]]

In [37]:
d, v = np.linalg.eig(x)
d = np.real(d)
v = np.real(v)
print(d)
print(d.dtype)

[1.15225668 0.25175166 0.25175166]
float64


In [38]:
d[0] = -13
d[d<0] = np.finfo(np.float64).eps
print(d)

[2.22044605e-16 2.51751661e-01 2.51751661e-01]


In [39]:
# from helps.projectPSD import projectPSD
# D = projectPSD(M)
# d = np.copy(D)
# print(d.shape)

In [40]:
# d[d<=0] = np.finfo(np.float64).eps
# for i in range(0, D.shape[0]):
#     print(D[i], d[i])

In [41]:
# from helps.projectPSD import projectPSD
# # d, V = projectPSD(M)
# M = projectPSD(M)
# # print(d.shape, V.shape)
# print(M.shape)
# print(np.diagonal(M))

In [42]:
M = torch.from_numpy(np.load('M.npy'))
idxtest = torch.from_numpy(np.load('idxtest.npy'))
idxtrain = torch.from_numpy(np.load('idxtrain.npy'))

In [43]:
torch.diagonal(M)

tensor([-342.5669,  293.3381,    0.0000,  ...,    0.0000,    0.0000,
        -120.3820], dtype=torch.float64)

In [44]:
from helps.projectPSD import projectPSD
M_ = torch.from_numpy(projectPSD(M))

In [45]:
torch.diagonal(M_)[:100]

tensor([9.4923e+02, 8.9512e+02, 2.2204e-16, 1.0735e+03, 8.2489e+02, 9.7003e+02,
        2.2204e-16, 8.7572e+02, 9.6203e+02, 2.2204e-16, 1.1507e+03, 9.2933e+02,
        9.0402e+02, 9.8201e+02, 1.0828e+03, 2.2204e-16, 2.2204e-16, 8.7227e+02,
        2.2204e-16, 9.8913e+02, 1.0689e+03, 2.2204e-16, 2.2204e-16, 2.2204e-16,
        9.0687e+02, 2.2204e-16, 2.2204e-16, 6.5718e+02, 2.2204e-16, 2.2204e-16,
        2.2204e-16, 2.2204e-16, 2.2204e-16, 2.2204e-16, 2.2204e-16, 1.1465e+03,
        2.2204e-16, 1.2879e+03, 1.1283e+03, 1.1166e+03, 2.2204e-16, 2.2204e-16,
        1.1469e+03, 2.2204e-16, 9.3253e+02, 8.8699e+02, 2.2204e-16, 2.2204e-16,
        8.7405e+02, 9.4235e+02, 2.2204e-16, 1.2192e+03, 9.3546e+02, 8.9555e+02,
        2.2204e-16, 1.0933e+03, 2.2204e-16, 1.0751e+03, 2.2204e-16, 1.2869e+03,
        8.3605e+02, 2.2204e-16, 1.1565e+03, 1.0362e+03, 8.4100e+02, 2.2204e-16,
        1.0771e+03, 2.2204e-16, 8.6608e+02, 7.1336e+02, 2.2204e-16, 2.2204e-16,
        2.2204e-16, 1.0066e+03, 2.2204e-

In [46]:
from helps.sqdist import mahalanobis_dist
data = K
p = data[:, idxa[idxtest]]
p = p.view(p.shape[0], -1)
q = data[:, idxb[idxtest]]
q = q.view(q.shape[0], -1)
# dist = mahalanobis_dist(p, q, M_)
A = M_

In [47]:
K = kernelmatrix(X=X, X2=torch.tensor([]), sigma=sigma)

In [48]:
K[0, 1]

tensor(0.4420, dtype=torch.float64)

In [49]:
Ap = torch.mm(A.double(), p)

In [50]:
A[0, 10]

tensor(-161.7417, dtype=torch.float64)

In [51]:
X[0, 0]

tensor(56.7630, dtype=torch.float64)

In [52]:
idxtest

tensor([309, 258, 603, 254, 464,  56,  30, 298, 367, 142, 128,  80, 575, 304,
        260,  78, 216,  79, 347, 134, 301, 478, 334, 541, 551, 136, 371, 542,
          2, 147,  99,  43, 117, 543,  16, 391, 627, 545, 404, 199, 480, 442,
        303, 228, 240,  23, 422, 305, 141,   9, 619, 380, 289, 599, 454, 423,
        594, 279, 139, 212, 188,  89, 353, 101,  29,  34, 374, 124, 329, 359,
        513, 221,  25, 346,  36, 327, 506, 311, 405, 297, 257, 537, 370, 624,
        466, 325, 628, 266, 432, 433, 584, 316, 290, 378,  26, 192, 589, 416,
        159, 180, 246, 570, 414,  28, 467, 450, 295, 291, 218, 494, 270, 523,
         81,  58,  47, 312, 168, 244, 195, 360,  21, 226,   6, 472,  33, 386,
        288, 227, 397, 401, 400, 591, 277, 413, 527, 324, 439,  70, 593, 208,
        217, 364, 460, 349, 451, 418,  32, 438, 255, 157, 508, 588, 179, 426,
        622, 336, 264, 273, 598, 234, 151, 213, 173, 131, 424, 175, 602, 623,
        210, 565, 437,  46, 547, 318, 488, 275, 122, 201, 174, 5

In [74]:
p = data[:, idxa[idxtest]]
q = data[:, idxb[idxtest]]
print(q[0, 2])

tensor(0.4873, dtype=torch.float64)


In [54]:
Ap = torch.mm(A.double(), p)
Aq = torch.mm(A.double(), q)

In [55]:
pmag = torch.sum(p*Ap, 0, keepdim=True).double()
qmag = torch.sum(q*Aq, 0, keepdim=True).double()

In [56]:
m = torch.mm(torch.ones(pn, 1).double(), qmag) + torch.mm(pmag.T, torch.ones(1, qn).double()) - 2 * torch.mm(p.T, Aq)

In [57]:
m[0, 10]

tensor(299.8538, dtype=torch.float64)

In [58]:
# No problem in Ap, Aq, pmag, qmag as well as M, K thus check for 2 lines above code or sqdist line 23 code 

In [59]:
dist = mahalanobis_dist(p, q, A)

In [60]:
dist[0, 11], m[0, 11]

(tensor(411.9349, dtype=torch.float64), tensor(411.9349, dtype=torch.float64))

In [76]:
from helps.sqdist import mahalanobis_dist
x = mahalanobis_dist(p, q, A)
x[0, 11]

tensor(411.9349, dtype=torch.float64)

### thus by far now sqdist is also working properly, there was a small writing error, wrote q instead of Aq in 2*p'*Aq section 

In [84]:
dist = x

In [97]:
tmp = 0
result = torch.zeros(1, dist.shape[1])
for pairCounter in range(0, dist.shape[1]):
    distPair = dist[pairCounter, :]
    tmp, idx = torch.sort(distPair)
    result[:, idx==pairCounter] = result[:, idx==pairCounter] + 1

tmp = 0
for counter in range(0, result.shape[1]):
    result[:, counter] = result[:, counter] + tmp
    tmp = result[:, counter]

In [100]:
tmp = 0
for counter in range(0, result.shape[1]):
    print(result[:, counter])

tensor([69.])
tensor([23.])
tensor([19.])
tensor([11.])
tensor([11.])
tensor([5.])
tensor([6.])
tensor([4.])
tensor([9.])
tensor([6.])
tensor([11.])
tensor([1.])
tensor([6.])
tensor([3.])
tensor([7.])
tensor([4.])
tensor([6.])
tensor([2.])
tensor([1.])
tensor([1.])
tensor([3.])
tensor([3.])
tensor([3.])
tensor([2.])
tensor([3.])
tensor([5.])
tensor([3.])
tensor([2.])
tensor([2.])
tensor([0.])
tensor([2.])
tensor([4.])
tensor([3.])
tensor([3.])
tensor([3.])
tensor([2.])
tensor([4.])
tensor([3.])
tensor([1.])
tensor([1.])
tensor([0.])
tensor([1.])
tensor([1.])
tensor([0.])
tensor([1.])
tensor([1.])
tensor([0.])
tensor([1.])
tensor([1.])
tensor([0.])
tensor([0.])
tensor([1.])
tensor([3.])
tensor([1.])
tensor([1.])
tensor([2.])
tensor([0.])
tensor([0.])
tensor([0.])
tensor([0.])
tensor([0.])
tensor([0.])
tensor([0.])
tensor([0.])
tensor([1.])
tensor([0.])
tensor([4.])
tensor([2.])
tensor([2.])
tensor([0.])
tensor([0.])
tensor([3.])
tensor([0.])
tensor([0.])
tensor([0.])
tensor([1.])
tensor